In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import torch
import torch.nn as nn
from torch.nn import functional as F
from torchinfo import summary

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

class SingleInputNet(nn.Module):
    """ Simple CNN model. """
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [4]:

model = SingleInputNet()
input_size = (2, 1, 28, 28)

input_tensors = torch.randn(input_size, device=torch.device('cuda'))

In [7]:
summary_list, stats = summary(model, input_size, verbose=3)


In [12]:
s1 = summary_list[0]

In [14]:
layer_props = [n for n in dir(summary_list[0]) if not n.startswith('__')]

In [15]:
layer_props

['calculate_macs',
 'calculate_num_params',
 'calculate_size',
 'check_recursive',
 'children',
 'class_name',
 'contains_lazy_param',
 'depth',
 'depth_index',
 'executed',
 'get_kernel_size',
 'get_layer_name',
 'get_param_count',
 'inner_layers',
 'input_size',
 'is_leaf_layer',
 'is_recursive',
 'kernel_size',
 'layer_id',
 'leftover_params',
 'leftover_trainable_params',
 'macs',
 'macs_to_str',
 'module',
 'num_params',
 'num_params_to_str',
 'output_bytes',
 'output_size',
 'param_bytes',
 'params_percent',
 'parent_info',
 'trainable',
 'trainable_params',
 'var_name']

In [19]:
for layer in summary_list:
    print(layer.module, layer.class_name, layer.children, layer.is_leaf_layer, layer.num_params, layer.depth, layer.depth_index, layer.input_size, layer.output_size)

SingleInputNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.3, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
) SingleInputNet [Conv2d: 1, Conv2d: 1, Dropout2d: 1, Linear: 1, Linear: 1] False 21840 0 1 [2, 1, 28, 28] [2, 10]
Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1)) Conv2d [] True 260 1 1 [2, 1, 28, 28] [2, 10, 24, 24]
Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1)) Conv2d [] True 5020 1 2 [2, 10, 12, 12] [2, 20, 8, 8]
Dropout2d(p=0.3, inplace=False) Dropout2d [] True 0 1 3 [2, 20, 8, 8] [2, 20, 8, 8]
Linear(in_features=320, out_features=50, bias=True) Linear [] True 16050 1 4 [2, 320] [2, 50]
Linear(in_features=50, out_features=10, bias=True) Linear [] True 510 1 5 [2, 50] [2, 10]
